In [10]:
from model import Model
import torch
torch.backends.cudnn.benchmark=True
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import argparse
import time
import numpy as np
import subprocess
from numpy import random
import copy
device="cpu"
from tqdm import tqdm

# import matplotlib.pyplot as plt
from data_loader import genomics_data

In [11]:
def kaiming_normal_init(m):
	if isinstance(m, nn.Conv2d):
		nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
	elif isinstance(m, nn.Linear):
		nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')


class Model(nn.Module):
	def __init__(self, classes):
		# Hyper Parameters
		self.init_lr = 0.1
		self.num_epochs = 40
		self.batch_size = 48
		
		self.pretrained = False
		self.momentum = 0.9
		self.weight_decay = 0.0001
		# Constant to provide numerical stability while normalizing
		self.epsilon = 1e-16

		# Network architecture
		super(Model, self).__init__()
		model = nn.Sequential(
		nn.Conv1d(1, 6, kernel_size=6),
		nn.ReLU(),
		nn.Conv1d(6, 3, kernel_size=6),
		nn.ReLU(),
		nn.Flatten(),
		nn.Linear(12258,1024),
		nn.ReLU()
		)

		self.feature_extractor=model

		self.feature_extractor.apply(kaiming_normal_init)
		self.fc = nn.Linear(1024, classes, bias=False)

		self.n_classes = 0

	def forward(self, x):
		x = self.feature_extractor(x)
		x = x.view(x.size(0), -1)
		x = self.fc(x)
		return x



	def classify(self, images):
		"""Classify images by softmax

		Args:
			x: input image batch
		Returns:
			preds: Tensor of size (batch_size,)
		"""
		_, preds = torch.max(torch.softmax(self.forward(images), dim=1), dim=1, keepdim=False)

		return preds

	def update(self, dataset, class_map):

		self.compute_means = True

		# Save a copy to compute distillation outputs


		classes = list(set(dataset.train_labels))

		loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size,
											   shuffle=True, num_workers=8)

		print("Batch Size (for n_classes classes) : ", len(dataset))
		optimizer = optim.SGD(self.parameters(), lr=self.init_lr, momentum = self.momentum, weight_decay=self.weight_decay)

		with tqdm(total=self.num_epochs) as pbar:
			for epoch in range(self.num_epochs):
				
				for i, (indices, images, labels) in enumerate(loader):
					seen_labels = []
					images = Variable(torch.FloatTensor(images)).to(device)
					seen_labels = torch.LongTensor([class_map[label] for label in labels.numpy()])
					labels = Variable(seen_labels).to(device)
					# indices = indices.cuda()

					optimizer.zero_grad()
					logits = self.forward(images)

					loss = nn.CrossEntropyLoss()(logits, labels)

					loss.backward()
					optimizer.step()

					if (i+1) % 1 == 0:
						tqdm.write('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
							   %(epoch+1, self.num_epochs, i+1, np.ceil(len(dataset)/self.batch_size), loss.data))

				pbar.update(1)

In [12]:
model = Model(66)
model.to(device)
all_classes = np.arange(66)

train_set = genomics_data(train=True,classes=all_classes)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=48,
													shuffle=True, num_workers=8)

test_set = genomics_data(train=False,classes=all_classes)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=48,
                                            shuffle=False, num_workers=8)


model.update(train_set, all_classes)

Batch Size (for n_classes classes) :  6947


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1/40], Iter [1/145] Loss: 4.1855
Epoch [1/40], Iter [2/145] Loss: 4.1856
Epoch [1/40], Iter [3/145] Loss: 4.1348


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1/40], Iter [4/145] Loss: 4.0267
Epoch [1/40], Iter [5/145] Loss: 3.5059
Epoch [1/40], Iter [6/145] Loss: 5.8337


...

 10%|█         | 4/40 [00:50<06:54, 11.52s/it]

Epoch [5/40], Iter [55/145] Loss: 3.8145
Epoch [5/40], Iter [56/145] Loss: 4.1437
Epoch [5/40], Iter [57/145] Loss: 3.7243


 10%|█         | 4/40 [00:50<06:54, 11.52s/it]

Epoch [5/40], Iter [58/145] Loss: 3.8507
Epoch [5/40], Iter [59/145] Loss: 3.6471
Epoch [5/40], Iter [60/145] Loss: 4.0803


 10%|█         | 4/40 [00:51<06:54, 11.52s/it]